In [37]:
import cv2
import numpy as np
import torch
import dlib
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
detector = dlib.get_frontal_face_detector()
model = torch.load("FirstTrainFaceNet.pth")

C:\Users\celik\AppData\Local\Temp\ipykernel_7716\1410913611.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("FirstTrainFaceNet.pth")


In [7]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
model.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=262144, out_features=256, bias=True)
  (14): ReLU()
  (15): Dropout(p=0.5, inplace=False)
  (16): Linear(in_features=256, out_features=128, bias=T

In [5]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [6]:
traindataset = datasets.ImageFolder(root="images/", transform=transform)
testdataset = datasets.ImageFolder(root="test/", transform=transform)

train_dataloader = DataLoader(traindataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(testdataset, batch_size=8)

In [20]:
def recognize(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_roi = img[y:y+h, x:x+w]
        resized = cv2.resize(face_roi, (256, 256))
        normalize = resized / 255.0
        reshaped = np.reshape(normalize, (1, 256, 256, 3))
        model.eval()
        result = model(reshaped)
        _, label = torch.max(result, 1)
        predicted = label[0].item()
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 47), 2)
        cv2.putText(img, str(predicted), (x+5, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255, 2))
    return img

In [46]:
label_to_name = {
    0: "Furkan",  # 0 etiketinin ismi
    1: "Kubilay"    # 1 etiketinin ismi
}

In [60]:
def recognize2(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_roi = img[y:y+h, x:x+w]
        
        # Görüntüyü yeniden boyutlandır
        resized = cv2.resize(face_roi, (256, 256))
        normalize = resized / 255.0
        reshaped = np.reshape(normalize, (1, 3, 256, 256))  # (1, 3, 256, 256) formatında

        # Tensor'a dönüştür
        tensor_image = torch.tensor(reshaped, dtype=torch.float32)
        
        # CUDA'ya taşı (eğer varsa)
        if torch.cuda.is_available():
            tensor_image = tensor_image.cuda()
            model.cuda()

        model.eval()  # Modeli değerlendirme moduna al

        with torch.no_grad():  # Gradient hesaplamalarını devre dışı bırak
            result = model(tensor_image)

        _, label = torch.max(result, 1)
        predicted_label = label[0].item()
        
        # Etiket numarasını isme dönüştür
        predicted_name = label_to_name.get(predicted_label, "Bilinmeyen")  # Bilinmeyen isim için varsayılan değer

        # Yüzün etrafına dikdörtgen çiz
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 47), 2)
        cv2.putText(img, predicted_name, (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)

    return img

In [63]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    # eğer baret takmıyorsa recognize yapacak
    recognize2(frame)
    cv2.imshow("Result", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [59]:
cap.release()
cv2.destroyAllWindows()

In [36]:
# Verilen DataLoader'dan bir sınıfa ait bir görüntüyü almak için bir fonksiyon
def show_sample_image(trainloader, class_index):
    
    # DataLoader'dan bir örnek al
    for images, labels in trainloader:
        # Sınıfı filtrele
        class_mask = (labels == class_index).nonzero(as_tuple=True)[0]
        if len(class_mask) > 0:
            sample_image = images[class_mask[0]]  # 1. örneği al
            sample_label = labels[class_mask[0]]   # Etiketini al
            break

     # Görüntüyü numpy dizisine çevir
    sample_image = sample_image.numpy().transpose((1, 2, 0))  # (C, H, W) -> (H, W, C)
    
    # Görüntüyü normalleştir (0-255 aralığına döndür)
    sample_image = (sample_image * 255).astype(np.uint8)

    # Görüntüyü görselleştir
    plt.imshow(sample_image)
    plt.title(f'Label: {sample_label.item()}')
    plt.axis('off')  # Eksenleri kapat
    plt.show()

In [35]:
# Örnek kullanım
class_index = 0  # İlgili sınıf indeksini belirtin
show_sample_image(train_dataloader, class_index)

ModuleNotFoundError: No module named 'matplotlib'

In [45]:
train_dataloader.dataset.classes

['furkan', 'kubilay']